In [ ]:
# import sys
# sys.path.append('F:\Kontiki\jupyter_webapp_env\Lib\site-packages')

In [ ]:
import pymongo
import warnings
warnings.filterwarnings('ignore')
import codecs
import base64
import hashlib
import io
import ipywidgets as widgets
import pandas as pd
import time
from datetime import datetime
from os.path import isfile, join
from IPython.display import HTML, display, clear_output

In [ ]:
# myclient = pymongo.MongoClient('mongodb://localhost:27017/')
# conn_str = "mongodb+srv://nyoman:<>@cluster0.uqnuh.mongodb.net/demonext?retryWrites=true&w=majority"
# myclient = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
# mydb = myclient['demonext']
col_response_false_deriv = 0

In [ ]:
output = widgets.Output()
df_raw = pd.read_excel('spurious.xlsx')
df_raw.sort_values('introduced_extent_ratio', inplace=True, ignore_index=True)
df = df_raw[df_raw['parent_node_count'] > 2]
df.reset_index(inplace=True)
list_response = list()
list1 = df['parent'].str.replace('.dot', '.png')
list2 = df['parent'].str.split().str.get(0).str.cat(['_'] * df.shape[0]).str.cat(df['child'].str.split().str.get(0)).str.cat(['.png'] * df.shape[0])
list2_title = df['child'].str.replace('.dot', '.png')
df['parent_child'] = list2
model_folder = 'reference'
spurious_folder = 'spurious'
counter = 0

In [ ]:
green_box_layout = widgets.Layout()
green_box_layout.width = '50 px'
green_box_layout.height = '100 px'
green_box_layout.border = '2px solid'


def fill_images(input_counter):
    h_page.value = '<{size}>{current} / {total}</{size}>'.format(size='h3', current=str(input_counter+1), total=str(df.shape[0]))
    model_file = open(join(model_folder, list1[input_counter]), 'rb')
    ib1.value = model_file.read()
    h1.value = list1[input_counter] + '<br />Graph shared by ' + str(df.loc[input_counter, 'parent_introduced_extent']) + ' families'
    question_file = open(join(spurious_folder, list2[input_counter]), 'rb')
    ib2.value = question_file.read()
    h2.value = list2_title[input_counter] + '<br />Graph shared by ' + str(df.loc[input_counter, 'child_introduced_extent']) + ' families'

def on_button_next_clicked(event):
    global counter
    res = {'concept1' : list2[counter].split('_')[0],
         'concept2' : list2[counter].split('_')[1].replace('.png', ''),
         'date' : datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
         'action' : str(validation_choice.value),
         'comment' : comment.value}
    col_response_false_deriv.insert_one(res)
    counter += 1
    validation_choice.value = None
    comment.value = ''
    if counter == df.shape[0]:
        counter -= 1
        return
    fill_images(counter)
    
def on_button_skip_clicked(event):
    global counter
    counter += 1
    validation_choice.value = None
    comment.value = ''
    if counter == df.shape[0]:
        counter -= 1
        return
    fill_images(counter)

def on_button_prev_clicked(event):
    global counter
    if counter == 0:
        return
    counter -= 1
    fill_images(counter)
    validation_choice.value = None
    comment.value = ''
    
link_download = widgets.HTML(value='')
   
hbox_layout = widgets.Layout()
hbox_layout.width = '100%'
hbox_layout.justify_content = 'space-around'
image_margin = '0 0 0 0'

boxes = []

model_file = open(join(model_folder, list1[counter]), 'rb')
h1_ref = widgets.HTML(value='<h3>Reference</h3>')
h1 = widgets.HTML(value=list1[counter] + '<br />Graph shared by ' + str(df.loc[counter, 'parent_introduced_extent']) + ' families')
ib1 = widgets.Image(value=model_file.read())
ib1.layout.object_fit = 'contain'
ib1.layout.margin = image_margin
boxb1 = widgets.Box()
boxb1.layout = green_box_layout
boxb1.children = [ib1]
vb1 = widgets.VBox()
vb1.layout.align_items = 'center'
vb1.children = [h1_ref, h1, boxb1]
boxes.append(vb1)

question_file = open(join(spurious_folder, list2[counter]), 'rb')
h2_quest = widgets.HTML(value='<h3>Question</h3>')
h2 = widgets.HTML(value=list2_title[counter] + '<br />Graph shared by ' + str(df.loc[counter, 'child_introduced_extent']) + ' families')
ib2 = widgets.Image(value=question_file.read())
ib2.layout.object_fit = 'contain'
ib2.layout.margin = image_margin
boxb2 = widgets.Box()
boxb2.layout = green_box_layout
boxb2.children = [ib2]
validation_choice = widgets.RadioButtons(
    options=[('Keep the red elements', 'Keep'), ('Delete the red elements', 'Delete'), ('Other', 'Other')],
    value=None,
    layout={'width': 'max-content'}, # If the items' names are long
    disabled=False
)
comment = widgets.Textarea(description='Comment:', placeholder='leave your comment here')
button_prev = widgets.Button(description='Skip & prev', tooltip='Skip this question and go to the previous question', style={'description_width': 'initial'})
button_prev.on_click(on_button_prev_clicked)
button_next = widgets.Button(description='Save & next', tooltip='Save answer and go to the next question', style={'description_width': 'initial'})
button_next.on_click(on_button_next_clicked)
button_skip = widgets.Button(description='Skip & next', tooltip='Skip this question and go to the next question', style={'description_width': 'initial'})
button_skip.on_click(on_button_skip_clicked)

next_skip = widgets.HBox()
next_skip.children = [button_prev, button_next, button_skip]

vb2 = widgets.VBox()
vb2.layout.align_items = 'center'
vb2.children = [h2_quest, h2, boxb2, validation_choice, comment, next_skip, link_download]
boxes.append(vb2)

vb = widgets.VBox()
h = widgets.HTML(value='<{size}>Validation of false derivations (in red)</{size}>'.format(size='h2'))
h_page = widgets.HTML(value='<{size}>{current} / {total}</{size}>'.format(size='h3', current=str(counter+1), total=str(df.shape[0])))
vb.layout.align_items = 'center'
hb = widgets.HBox()
hb.layout = hbox_layout
hb.children = boxes

In [ ]:
def on_button_goto_clicked(event):
    global counter
    counter = goto_input.value
    fill_images(counter)
    validation_choice.value = None
    comment.value = ''

goto_page_desc = widgets.HTML(value='Go to family')
goto_page = widgets.HBox()
goto_button = widgets.Button(description='go', layout=widgets.Layout(width='50px'))
goto_button.on_click(on_button_goto_clicked)
goto_input = widgets.Text(layout=widgets.Layout(width='50px'))
goto_options = list()
idx = 0
for l in list2_title:
    goto_options.append((l.split()[1].split('.')[0], idx))
    idx += 1
goto_options.sort(key=lambda tup: tup[0])
goto_input = widgets.Dropdown(options=goto_options, disabled=False)
goto_page.children = [goto_page_desc, goto_input, goto_button]

def on_button_gotonumber_clicked(event):
    global counter
    try:
        page_number_input = int(goto_number_input.value) - 1
        goto_number_input.value = ''
    except ValueError:
        goto_number_input.value = ''
        return
        
    if page_number_input >= df.shape[0] or page_number_input < 0:
        goto_number_input.value = ''
        return
    counter = page_number_input
    fill_images(counter)
    validation_choice.value = None
    comment.value = ''
    goto_number_input.value = ''

goto_number_desc = widgets.HTML(value='Go to page number')
goto_number_input = widgets.Text(layout={'width': '50px'}, disabled=False)
goto_number_button = widgets.Button(description='go', layout=widgets.Layout(width='50px'))
goto_number_button.on_click(on_button_gotonumber_clicked)
goto_number = widgets.HBox()
goto_number.children = [goto_number_desc, goto_number_input, goto_number_button]
goto = widgets.VBox()
goto.children = [goto_page, goto_number]

In [ ]:
footer = widgets.VBox()
footer_header = widgets.HTML(value='<{size}>Graph notations</{size}>'.format(size='h2'))
footer_content = widgets.HBox()
help_file = open('help.png', 'rb')
help_image = widgets.Image(value=help_file.read(), format='png')
help_image.layout.object_fit = 'contain'
edge_origin = widgets.HTML(value='Edge label: <code>[origin]: [construction1]-[construction2]</code><br /> \
                           <code>origin</code>: \
                           <ul> <li>C = converts</li> <li>D = demonette1</li> \
                           <li>Ma = dimocXaie</li> \
                           <li>Mb = dimocXat</li> <li>Mc = dimocXet</li> <li>Md = dimocXier</li> \
                           <li>Na = denomCONVX</li> <li>Nb = denomPREFX</li> <li>Nc = denomXaire</li> \
                           <li>Nd = denomXal</li> <li>Ne = denomXique</li> <li>Nf = denomXSUF1</li> \
                           <li>Ng = denomXSUF2</li> <li>Nh = denomXSUF3</li> <li>Ni = denomXSUF4</li> \
                           <li>Nj = denomXSUF5</li> <li>Nk = denomXSUF6</li> \
                           <li>Ra = derifantiX</li> <li>Rb = derifde1X</li> <li>Rc = derifenX</li> \
                           <li>Rd = derifinX</li> <li>Re = derifQUANTX</li> <li>Rf = derifreX</li> \
                           <li>Rg = deriftriX</li> <li>Rh = derifXable</li> <li>Ri = derifXiser</li> \
                           </ul> \
                           ')
footer_content.children = [edge_origin, help_image]
footer.children = [footer_header, footer_content]

In [ ]:
def on_login_button_clicked(event):
    global col_response_false_deriv
    conn_str = "mongodb+srv://" + username.value + ":" + password.value + \
    "@cluster0.uqnuh.mongodb.net/demonext?retryWrites=true&w=majority"
    try:
        myclient = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
        myclient.server_info()
        username.value = ''
        password.value = ''
        mydb = myclient['demonext']
        col_response_false_deriv = mydb['response_falsederiv']
        vb.children = [h, goto, h_page, hb, footer]
        invalid.value = 'success'
    except Exception as err:
        username.value = ''
        password.value = ''
        invalid.value = str(err)
    
login_form = widgets.VBox()
username = widgets.Text(placeholder='Enter username', description='Username:', disabled=False)
password = widgets.Password(placeholder='Enter password', description='Password:', disabled=False)
invalid = widgets.HTML(value='')
login_button = widgets.Button(description='Login', layout=widgets.Layout(width='100px'))
login_button.on_click(on_login_button_clicked)
login_footer = widgets.HBox()
login_footer.children = [login_button, invalid]
login_form.children = [username, password, login_footer]

In [ ]:
vb.children = [login_form] 
display(vb)